# Capstone Final Project 

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                           /                                      failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|4_73_pypy|3_73_pypy|2_73_pypy|0_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> python[versio

In [3]:
import requests 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [4]:
!pip install folium
import folium 

print('Libraries imported.')

     |████████████████████████████████| 94 kB 4.8 MB/s eta 0:00:011
Libraries imported.


Get the neighbourhood information of Toronto from Wikipedia.

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df_table= dfs[0]

Clean up the data. Get rid of the invaild data and group neighbourhood by postal code.

In [6]:
df= df_table[df_table['Borough']!='Not assigned']
df.Neighbourhood.replace('Not assigned',df.Borough,inplace = True)
df.groupby(['Postal Code']).agg(','.join)
df.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
df.shape

(103, 3)

Get the file of geographic coordinates of Toronoto neighbourhood

In [8]:
url_2 = 'http://cocl.us/Geospatial_data'
df_2 = pd.read_csv(url_2)
df_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the datafram into one.

In [25]:
df_geo = pd.merge(df,df_2,on='Postal Code')
df_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Define the URL

In [18]:
CLIENT_ID = 'V4MKX0ERS4ITCWSLGCGXZ2R105NC15JNWIKEH0COQBSP5OLW'
CLIENT_SECRET = 'LLMNRMPJBL31FTMP3GSMOG1AW2GK33TWXKJCC3DALCAL5RGB'
ACCESS_TOKEN = '2GKAWHI3WYR2Q4WMI253KXZJACJDB2PPPVCOTYDDGGSHDOUG'
VERSION = '20180605' 


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V4MKX0ERS4ITCWSLGCGXZ2R105NC15JNWIKEH0COQBSP5OLW
CLIENT_SECRET:LLMNRMPJBL31FTMP3GSMOG1AW2GK33TWXKJCC3DALCAL5RGB


Get nearby venues.

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT = 10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=df_geo['Neighbourhood'],
                                   latitudes=df_geo['Latitude'],
                                   longitudes=df_geo['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(9782, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
2,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
3,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store


Dog parks are categorized as Dog Run. Put all dog run locations into one dataframe.

In [48]:
toronto_dogrun= toronto_venues.loc[toronto_venues['Venue Category']== 'Dog Run']
toronto_dogrun

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
299,"Regent Park, Harbourfront",43.654260,-79.360636,Cherry Beach Off-Leash Dog Area,43.636727,-79.344335,Dog Run
717,Don Mills,43.745906,-79.352188,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
1247,Don Mills,43.725900,-79.340923,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
1679,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Doggie Central,43.623252,-79.575922,Dog Run
1870,The Beaches,43.676357,-79.293031,Kew Beach Off Leash Dog Park,43.665815,-79.296988,Dog Run
2207,Leaside,43.709060,-79.363452,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
2258,Leaside,43.709060,-79.363452,Dogs Off-Leash Area,43.716589,-79.384246,Dog Run
2534,Cedarbrae,43.773136,-79.239476,Thomson Memorial Dog Park,43.756253,-79.253294,Dog Run
2757,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Avondale Park,43.757053,-79.408127,Dog Run
2812,Thorncliffe Park,43.705369,-79.349372,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run


Put all dog pet stores into one dataframe.

In [39]:
toronto_petstore= toronto_venues.loc[toronto_venues['Venue Category']== 'Pet Store']
toronto_petstore

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
127,Victoria Village,43.725882,-79.315572,PetSmart,43.729945,-79.282226,Pet Store
357,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Parker Pet Care,43.723333,-79.474341,Pet Store
1209,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Pet Valu,43.780045,-79.137906,Pet Store
1287,Don Mills,43.725900,-79.340923,PetSmart,43.712682,-79.362636,Pet Store
2206,Leaside,43.709060,-79.363452,PetSmart,43.712682,-79.362636,Pet Store
2816,Thorncliffe Park,43.705369,-79.349372,PetSmart,43.712682,-79.362636,Pet Store
3680,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,PetSmart,43.729945,-79.282226,Pet Store
3945,Downsview,43.737473,-79.464763,Parker Pet Care,43.723333,-79.474341,Pet Store
3974,Downsview,43.737473,-79.464763,PetSmart,43.716544,-79.445926,Pet Store
4285,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,PetSmart,43.729945,-79.282226,Pet Store


We only choose neighbourhoods that does not have a pet store.

In [53]:
#get neighbourhoods with pet stores in lists
storelist=toronto_petstore['Neighbourhood'].tolist()
storelist

['Victoria Village',
 'Lawrence Manor, Lawrence Heights',
 'Rouge Hill, Port Union, Highland Creek',
 'Don Mills',
 'Leaside',
 'Thorncliffe Park',
 'Kennedy Park, Ionview, East Birchmount Park',
 'Downsview',
 'Downsview',
 'Golden Mile, Clairlea, Oakridge',
 'North Park, Maple Leaf Park, Upwood Park',
 'Willowdale, Newtonbrook',
 'Downsview',
 'Bedford Park, Lawrence Manor East',
 'Dorset Park, Wexford Heights, Scarborough Town Centre',
 'Davisville North',
 'Wexford, Maryvale',
 'Wexford, Maryvale',
 'Willowdale, Willowdale West',
 'Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens',
 'Davisville',
 "Clarks Corners, Tam O'Shanter, Sullivan",
 'Business reply mail Processing Centre, South Central Letter Processing Plant Toronto',
 "Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East"]

In [55]:
#neighbourhoods with pet stores in dataframe 
toronto_dogrun[toronto_dogrun['Neighbourhood'].isin(storelist)]

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
717,Don Mills,43.745906,-79.352188,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
1247,Don Mills,43.725900,-79.340923,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
2207,Leaside,43.709060,-79.363452,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
2258,Leaside,43.709060,-79.363452,Dogs Off-Leash Area,43.716589,-79.384246,Dog Run
2812,Thorncliffe Park,43.705369,-79.349372,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
5323,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Avondale Park,43.757053,-79.408127,Dog Run
6175,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,Thomson Memorial Dog Park,43.756253,-79.253294,Dog Run
6366,Davisville North,43.712751,-79.390197,Dogs Off-Leash Area,43.716589,-79.384246,Dog Run
6369,Davisville North,43.712751,-79.390197,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run
7565,Davisville,43.704324,-79.388790,Sunnybrook Dog Park,43.722474,-79.365727,Dog Run


In [68]:
#neighbourhoods that are not in the list above are the locations we want
df_location = toronto_dogrun[toronto_dogrun['Neighbourhood'].isin(storelist) == False]
df_location

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
299,"Regent Park, Harbourfront",43.654260,-79.360636,Cherry Beach Off-Leash Dog Area,43.636727,-79.344335,Dog Run
1679,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Doggie Central,43.623252,-79.575922,Dog Run
1870,The Beaches,43.676357,-79.293031,Kew Beach Off Leash Dog Park,43.665815,-79.296988,Dog Run
2534,Cedarbrae,43.773136,-79.239476,Thomson Memorial Dog Park,43.756253,-79.253294,Dog Run
2757,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Avondale Park,43.757053,-79.408127,Dog Run
3106,Scarborough Village,43.744734,-79.239476,Thomson Memorial Dog Park,43.756253,-79.253294,Dog Run
3421,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,Withrow Park Off Leash Dog Park,43.674019,-79.346764,Dog Run
3622,"Little Portugal, Trinity",43.647927,-79.419750,Stanley Park (Dog Park),43.641668,-79.409269,Dog Run
4268,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Stanley Park (Dog Park),43.641668,-79.409269,Dog Run
4449,"York Mills, Silver Hills",43.757490,-79.374714,Avondale Park,43.757053,-79.408127,Dog Run


In [66]:
df_location.shape

(26, 7)

There are 26 locations available to choose.

In [67]:
#Get the coordinates of the locations 
xy=df_location[['Venue Latitude', 'Venue Longitude']]
xy.head()

,Venue Latitude,Venue Longitude
299,43.636727,-79.344335
1679,43.623252,-79.575922
1870,43.665815,-79.296988
2534,43.756253,-79.253294
2757,43.757053,-79.408127


Put locations into 5 clusters, and each centre of the cluster is surrounded by the dog parks with shortest distance. The owner can search up store location from the centre of each clusters. 

In [101]:
from sklearn.cluster import KMeans

number_of_clusters = 5

kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(xy)

kmeanslabels = kmeans.labels_
kmeanslabels

array([4, 2, 4, 3, 0, 3, 4, 1, 1, 0, 4, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1], dtype=int32)

Get the cluster centres.

In [73]:
kmeans_clustercenters = kmeans.cluster_centers_
kmeans_clustercenters

array([[ 43.7372185 , -79.39955806],
       [ 43.64336046, -79.4371197 ],
       [ 43.62325204, -79.57592205],
       [ 43.75625332, -79.25329448],
       [ 43.65332219, -79.33310537]])

The coordinates of the five centres are shown below.

In [79]:
df_cc = pd.DataFrame({'lat': kmeans_clustercenters[:, 0], 'lng': kmeans_clustercenters[:, 1]})
df_cc

,lat,lng
0,43.737218,-79.399558
1,43.643360,-79.437120
2,43.623252,-79.575922
3,43.756253,-79.253294
4,43.653322,-79.333105


Let's visualize clusters on the map

In [77]:
#get Toronto Geographic Coordinates
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


Cluster Centres are shown in red spots and the dog parks nearby are shown in blue spots.

In [98]:
cluster_map = folium.Map(location=[43.6534817, -79.3839347], zoom_start=11)

for lat, lng in zip (df_cc.lat, df_cc.lng):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(cluster_map)

for lat1, lng1 in zip (df_location['Venue Latitude'], df_location['Venue Longitude']):
    folium.CircleMarker(
        [lat1, lng1],
        radius=3,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(cluster_map)

cluster_map